# 目标
- 计算物体的周长、面积、质心、最小外接矩形等
- OpenCV函数：cv2.contourArea(), cv2.arcLength(), cv2.approxPolyDP() 等

# 教程
在计算轮廓特征之前，我们先用上一节的代码把轮廓找到：
![](http://blog.codec.wang/cv2_31_handwriting_sample.jpg)

In [2]:
import cv2
import numpy as np

img = cv2.imread('handwriting.jpg', 0)
_, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
image, contours, hierarchy = cv2.findContours(thresh, 3, 2)

# 以数字3的轮廓为例
cnt = contours[0]

为了便于绘制，我们创建出两幅彩色图，并把轮廓画在第一幅图上：

In [3]:
img_color1 = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
img_color2 = np.copy(img_color1)
cv2.drawContours(img_color1, [cnt], 0, (0, 0, 255), 2)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

## 轮廓面积

In [5]:
area = cv2.contourArea(cnt)  # 4807.0
print(area)

4807.0


注意轮廓特征计算的结果并不等同于像素点的个数，而是根据几何方法算出来的，所以有小数。

如果统计二值图中像素点个数，应尽量避免循环，可以使用cv2.countNonZero()，更加高效。

## 轮廓周长

In [7]:
perimeter = cv2.arcLength(cnt, True)  # 612.7838320732117
print(perimeter)

612.7838320732117


参数2表示轮廓是否封闭，显然我们的轮廓是封闭的，所以是True。

## 图像矩
矩可以理解为图像的各类几何特征，详情请参考：[Image Moments]

In [8]:
M = cv2.moments(cnt)
print(M)

{'m00': 4807.0, 'm10': 849431.6666666666, 'm01': 567076.0, 'm20': 152202834.8333333, 'm11': 98742806.83333333, 'm02': 78411253.5, 'm30': 27622130168.100002, 'm21': 17483376460.433334, 'm12': 13393681126.366667, 'm03': 12075225547.800001, 'mu20': 2102115.8118831217, 'mu11': -1463623.748457104, 'mu02': 11513980.819325984, 'mu30': -16129690.252017975, 'mu21': 45457166.5082975, 'mu12': -116830615.75261116, 'mu03': 108563520.76313782, 'nu20': 0.09097213657424653, 'nu11': -0.06334045859189485, 'nu02': 0.4982843617310733, 'nu30': -0.010067942397799047, 'nu21': 0.028373770780591508, 'nu12': -0.07292414741501461, 'nu03': 0.06776393448775536}


M中包含了很多轮廓的特征信息，比如M[‘m00’]表示轮廓面积，与前面cv2.contourArea()计算结果是一样的。质心也可以用它来算：

In [9]:
cx, cy = M['m10'] / M['m00'], M['m01'] / M['m00']  # (176.70723250814783, 117.96879550655294)
print((cx,cy))

(176.70723250814783, 117.96879550655294)


## 外接矩形
形状的外接矩形有两种，如下图，绿色的叫外接矩形，表示不考虑旋转并且能包含整个轮廓的矩形。蓝色的叫最小外接矩，考虑了旋转：
![](http://blog.codec.wang/cv2_min_rect_rect_bounding.jpg)

In [11]:
x, y, w, h = cv2.boundingRect(cnt)
print((x, y, w, h))
cv2.rectangle(img_color1, (x, y), (x+w, y+h), (0, 255, 0), 2)

rect = cv2.minAreaRect(cnt)
print(rect)
box = np.int0(cv2.boxPoints(rect))
print(box)
cv2.drawContours(img_color1, [box], 0, (255, 0, 0), 2)

cv2.imshow('img', img_color1)
cv2.waitKey(0)

(124, 40, 93, 165)
((174.4885711669922, 124.240478515625), (169.71473693847656, 79.99760437011719), -81.20945739746094)
[[201 214]
 [121 201]
 [147  34]
 [226  46]]


-1

其中np.int0(x)是把x取整的操作，比如377.93就会变成377，也可以用x.astype(np.int)。

## 最小外接圆
外接圆跟外接矩形一样，找到一个能包围物体的最小圆

In [12]:
(x, y), radius = cv2.minEnclosingCircle(cnt)
(x, y, radius) = np.int0((x, y, radius))  # 圆心和半径取整
cv2.circle(img_color2, (x, y), radius, (0, 0, 255), 2)


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

![](http://blog.codec.wang/cv2_min_enclosing_circle.jpg)

## 拟合椭圆
我们可以用得到的轮廓拟合出一个椭圆：

In [ ]:
ellipse = cv2.fitEllipse(cnt)
cv2.ellipse(img_color2, ellipse, (255, 255, 0), 2)


![](http://blog.codec.wang/cv2_fitting_ellipse.jpg)

## 形状匹配
cv2.matchShapes()可以检测两个形状之间的相似度，返回值越小，越相似。先读入下面这张图片：
![](http://blog.codec.wang/cv2_match_shape_shapes.jpg)

In [13]:
img = cv2.imread('shapes.jpg', 0)
_, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
image, contours, hierarchy = cv2.findContours(thresh, 3, 2)
img_color = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR) # 用于c

图中有3条轮廓，我们用A/B/C表示：

In [14]:
cnt_a, cnt_b, cnt_c = contours[0], contours[1], contours[2]
print(cv2.matchShapes(cnt_b, cnt_b, 1, 0.0))
print(cv2.matchShapes(cnt_b, cnt_c, 1, 0.0))
print(cv2.matchShapes(cnt_b, cnt_a, 1, 0.0))

0.0
2.176955476018705e-05
0.4181711500194456


可以看到BC相似程度比AB高很多，并且图形的旋转或缩放并没有影响。

cv2.matchShapes()其中，参数3是匹配方法，详情可参考：ShapeMatchModes，参数4是OpenCV的预留参数，暂时没有实现，可以不用理会。

形状匹配是通过图像的Hu矩来实现的(cv2.HuMoments())，大家如果感兴趣，可以参考：Hu-Moments
